# Getting Parameters and Passing to SQL

Now let's see how we can query for the status of a Citibike station over time.

For this part, we want to create a new function, where we will pass the `station_id` as a **parameter**. Then our code will read the value of the parameter `station_id` and then will query the database to get the status of that station.

The related pieces of code for reading a parameter are the following

> `from flask import request`

> `station_id = request.args.get('station_id')`

Now, let's add the import command `from flask import request` into the `webserver.py` and also add the function below that adds an extra URL `station_status`. 

```python
from flask import request
@app.route('/station_status')
def station_status():

    station_id = int(request.args.get('station_id'))

    conn_string = 'mysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
        host = 'db.ipeirotis.org', 
        user = 'student',
        db = 'citibike_fall2017',
        password = 'dwdstudent2015',
        encoding = 'utf8mb4')

    engine = create_engine(conn_string)
    con = engine.connect()
    query = '''SELECT available_bikes, 
                      available_docks, 
                      capacity, 
                      available_bikes / capacity AS percent_full,
                      communication_time 
               FROM status_fall2017
               WHERE id = %s'''
    status = con.execute(query, (station_id,))

    con.close()

    return render_template('station_status.html', station_id = station_id, statuses=status)
```

And we can create the corresponding template file under `templates/station_status.html` for showing the results:
    
```html
<!DOCTYPE html>
<html>
<body> 
    <table>
        <thead>
            <tr>    <!-- Construct table header -->
                    <th> Time </th>
                    <th> Bikes available </th>
                    <th> Docks available </th>
                    <th> Total docks </th>
                    <th> Percent full </th>
            </tr>
        </thead>
        <tbody>
            {% for status in statuses %}
                <tr>
                    <td>{{ status.communication_time }}</td>
                    <td>{{ status.available_bikes }}</td>
                    <td>{{ status.available_docks }}</td>
                    <td>{{ status.capacity }}</td>
                    <td>{{ status.percent_full }}</td>
                </tr>
            {% endfor %}
         </tbody>
    </table>
</body>
</html>
```

This function creates a new URL, to which we can pass a parameter `station_id` as follows:

> http://your_ip_address:5000/station_status?station_id=72



#### Now let's start our server:



```bash
$  python3 webserver.py
```